# Discriminative Biclustering Algorithm 
Proposed by Odibat & Reddy, 2014 in **Efficient mining of discriminative co-clusters from gene
expression data**

In [2]:
%load_ext pycodestyle_magic

In [3]:
%matplotlib inline
import numpy as np
import math
from matplotlib import pyplot as plt
from sklearn.metrics import consensus_score

In [4]:
# !pip install pycodestyle
# !pip install pycodestyle_magic

### Definition 1 - Coherence Measure H

In [5]:
#%pycodestyle
class CoherenceMeasure(object):
    def __init__(self, data):
        self.data = data
        self.n, self.m = data.shape
        self.xiJ = np.mean(data, axis=1)
        self.xIj = np.mean(data, axis=0)
        self.xIJ = np.mean(data)
        self._H = None

    @property
    def H(self):
        if self._H is None:
            print("Computing coherence measure")
            self._H = self._compute_H()
            print("H value: " + str(self._H))
        return self._H
    
    def _compute_H(self):
        H = 0
        for i in range(self.n):
            for j in range(self.m):
                H += (self.data[i, j] - self.xIj[j] -
                      self.xiJ[i] + self.xIJ)**2
        H *= 1.0/math.fabs(self.m*self.n)
        H = 1 - H
        return H

#### Loading test data for Coherence Measure

In [6]:
import random
data = np.random.random((50, 50))
print(data)

[[ 0.99292617  0.46054783  0.45334732 ...,  0.54637177  0.34975704
   0.03320745]
 [ 0.4967808   0.92398397  0.850448   ...,  0.8214379   0.99155833
   0.44774937]
 [ 0.60443283  0.04417372  0.73247088 ...,  0.66486059  0.44419794
   0.35091846]
 ..., 
 [ 0.83547912  0.1982444   0.91010615 ...,  0.63991897  0.3788043
   0.3138958 ]
 [ 0.36928451  0.08344092  0.87638898 ...,  0.9969633   0.40814035
   0.36725078]
 [ 0.48100482  0.85805674  0.31450355 ...,  0.10528504  0.85085786
   0.72156388]]


In [7]:
# Testing Coherence
coherence_measure = CoherenceMeasure(data)
print("H = " + str(coherence_measure.H))

Computing coherence measure
H value: 0.920836263328
H = 0.920836263328


### Definition 2 - Positive and negative correlations

In [8]:
# input: rows x and y and J columns
# output: positive and negative correlations


class PositiveNegativeCorrelation(object):
    def __init__(self, x, y, J):
        self._x = x
        self._y = y
        self._J = J
        self._x_mean = np.mean(x)
        self._y_mean = np.mean(y)
        self._H_pos = None
        self._H_neg = None

    @property
    def H_pos(self):
        if self._H_pos is None:
            # print("Computing H positive...")
            self._H_pos = self._compute_H_pos()
            # print("H positive value: " + str(self._H_pos))
        return self._H_pos

    @property
    def H_neg(self):
        if self._H_neg is None:
            # print("Computing H negative...")
            self._H_neg = self._compute_H_neg()
            # print("H negative value: " + str(self._H_neg))
        return self._H_neg

    def _compute_H_pos(self):
        H_pos = 0
        for j in range(self._J):
            aux = (((self._x[j] - self._x_mean) -
                    (self._y[j] - self._y_mean))/2.0)**2
            H_pos += aux
        H_pos *= 1.0/math.fabs(self._J)
        H_pos = 1 - H_pos
        return H_pos

    def _compute_H_neg(self):
        H_neg = 0
        for j in range(self._J):
            aux = (((self._x[j] - self._x_mean) +
                    (self._y[j] - self._y_mean))/2.0)**2
            H_neg += aux
        H_neg *= 1.0/math.fabs(self._J)
        H_neg = 1 - H_neg
        return H_neg

#### Loading test data for positive and negative correlation

In [9]:
x = np.random.random((5))
y = np.random.random((5))
J = 5
print("Row x " + str(x))
print("Row y " + str(y))
print("J value " + str(J))

Row x [ 0.35116499  0.61168243  0.54373779  0.24584837  0.53652892]
Row y [ 0.44920898  0.25643547  0.79416032  0.92575588  0.77734419]
J value 5


In [10]:
# Testing correlation
positive_negative_correlation = PositiveNegativeCorrelation(x,y,J)
print("H positive " + str(positive_negative_correlation.H_pos))
print()
print("H negative " + str(positive_negative_correlation.H_neg))

H positive 0.972413371625

H negative 0.987450189092


### Definition 3 - Pair-based coherence

In [193]:
#%%pycodestyle

# input: co-cluster X of I rows and J columns
# output: paired-based coherence


class PairBasedCoherence(object):
    def __init__(self, X):
        self._X = np.array(X)
        self._I, self._J = X.shape
        self._HP = None

    @property
    def HP(self):
        if self._HP is None:
            # print("Calculating Pair based coherence..")
            self._HP = self._compute_HP_()
            print("Paired based coherence value: " + str(self._HP))
        return self._HP

    def _compute_HP_(self):
        HP = 0
        for i in range(self._I):
            for j in range(i+1, self._I):
                if (i==j): 
                    break
                x = self._X[i]
                y = self._X[j]
                correlation = PositiveNegativeCorrelation(x, y,self._J)
                # H0 = correlation.H_pos + correlation.H_neg
                H0 = max(correlation.H_pos,correlation.H_neg)
                HP += H0
        HP *= math.fabs(2.0)/(math.fabs(self._I)*(math.fabs(self._I)-1))
        return HP

#### Loading test data for pair-based coherence

In [171]:
data = np.random.random((50, 50))
print(data)

[[ 0.35201519  0.66624139  0.47324458 ...,  0.28920509  0.48218676
   0.19888243]
 [ 0.99267477  0.85729539  0.58811335 ...,  0.84362019  0.47761887
   0.7530935 ]
 [ 0.7929525   0.25019647  0.78238322 ...,  0.90085513  0.41874252
   0.8099857 ]
 ..., 
 [ 0.46498502  0.02534155  0.5364507  ...,  0.12980135  0.24798783
   0.40901146]
 [ 0.054231    0.88829901  0.24406213 ...,  0.8294805   0.24632265
   0.05511035]
 [ 0.66915714  0.17731626  0.49298928 ...,  0.5113613   0.52655005
   0.49508702]]


In [172]:
pair_based_coherence = PairBasedCoherence(data)
print("H value " + str(pair_based_coherence.HP))

Calculating Pair based coherence..
Paired based coherence value: 1.91836331183
H value 1.91836331183


### Coherence for a new z in in X

In [14]:
%%latex
Define H for a new term 'z' in X
$$
H_{1}(I,J,X,z) =H_{0}(I,J,X) \cdot \frac{(I-1)}{(I+1)} + \frac{|2|}{(I)(I+1)} \cdot \sum_{x \epsilon X} {h(x,z,J)}
$$

<IPython.core.display.Latex object>

### RAPOOC

This algorithm is proposed to efficiently extract the most coherent and large co-clusters that area arbitrarily positioned in the data matrix.



#### Algorithm 1 RAPOOC (D,k,l,K)
Input: Data matrix D, number of row clusters (k), number of column clusters (l), number of optimized co-clusters (K)

Output: A set of K co-clusters({X})

In [267]:
# https://github.com/munikarmanish/kmeans/blob/master/kmeans.py


class BisectingClusterer(object):
    def __init__(self, data):
        if data is not None:
            self._data = np.array(data)
            self._I, self._J = self._data.shape
        else:
            print("Empty data")
    
    @property
    def centroids(self):
        return self._centroids

    def fit(self):
        self._centroids = self._compute_centroids_()
        bisecting_indices = self._bisect_clusters_(self._centroids)
        return bisecting_indices
    
    def _compute_centroids_(self):
        max_correlation = 0
        centroids = [0,0]
        for i in range(self._I):
            for j in range(i+1, self._I):
                if (i == j):
                    break
                correlation = PositiveNegativeCorrelation(self._data[i],
                                                          self._data[j],
                                                          self._J).H_neg
                if(correlation > max_correlation):
                    max_correlation = correlation
                    centroids[0] = i
                    centroids[1] = j
        return centroids

    def _bisect_clusters_(self, centroids):
        cluster_indices = np.zeros(self._I)
        for i in range(self._I):
            correlation0 = PositiveNegativeCorrelation(
                self._data[centroids[0]], self._data[i],self._J).H_pos
            correlation1 = PositiveNegativeCorrelation(
                self._data[centroids[1]], self._data[i],self._J).H_pos
            if(correlation0 <= correlation1):
                cluster_indices[i] = 1
        return cluster_indices
    

In [276]:
#%%pycodestyle


class Rapooc(object):
    def __init__(self, D, k, l, K):
        self._D = np.array(D)
        self._k = k
        self._l = l
        self._K = K
        self._rho = np.ones(D.shape[0])
        self._gamma = np.ones(D.shape[1])

    def initialize(self):
        i = 1
        j = 1
        while (i < self._k or j < self._l):
            if i < self._k:
                i += 1
                print('\n0. argmin\n')
                alpha = self._argmin_H_(self._rho, self._gamma,i)
                print('\n4. alpha: ' + str(alpha) + '\n ======= \n')
                print('% debug % '+ str(len(self._D[np.where(self._rho==alpha)])))
                self._bisect_partitions_(self._D[np.where(self._rho==alpha)], self._rho, alpha, i)
            '''
            if j < l:
                j += 1
                beta = self._argmin_H(self._rho,self._gaamma)
                self._bisect_partitions_(self._D[np.where(self._rho==alpha)],np.where(self._rho==alpha),i)
            '''
            j += 1 
            
    def _argmin_H_(self, row_co_cluster, col_co_cluster,num_coclusters,option='row'):
        if (option=='row'):
            data = self._D
        else:
            data = self._D.T
        h_min = math.inf
        min_cocluster = 1
        print('1. calculating arg min\n')
        print('2. coclusters: ' + str(np.max(np.int64(row_co_cluster))))
        for i in range(1, np.max(np.int64(row_co_cluster))):
            coherence = PairBasedCoherence(
                data[np.where(row_co_cluster == i)]).HP # self._D[np.where(row_co_cluster == i)]
            if (coherence <= h_min):
                h_min = coherence
                min_cocluster = i
        print ('3. coherence: ' + str(h_min) + '\n')
        return min_cocluster

    def _bisect_partitions_ (self, data, mapping_array, cluster_to_replace, new_cluster_index):
        clusterer = BisectingClusterer(data)
        bisected_map = clusterer.fit()
        bisected_map_index = 0
        for i in range(0,cluster_to_replace):
            if (mapping_array[i] == cluster_to_replace): 
                if (bisected_map[bisected_map_index] == 1):
                    mapping_array[i] = new_cluster_index
                new_cluster_index += 1

In [277]:
import pandas as pd
import glob as glob


data = pd.read_csv('TestData/SimulatedDataCoherence/LowCoherenceMix.csv',header=None)
data.head()

,0,1,2,3,4,5,6
0,0.335750,0.087721,0.48434,0.70389,0.28692,0.317500,0.28815
1,0.318860,0.132890,0.72690,0.54530,0.48470,0.126590,0.48244
2,0.148530,0.131420,0.69661,0.61358,0.26988,0.074009,0.48441
3,0.139390,0.016388,0.38919,0.92711,0.57897,0.001382,0.49239
4,0.085258,0.192790,0.40184,0.80296,0.60293,0.004399,0.21951


In [278]:
rapooc = Rapooc(data,4,4,2)
rapooc.initialize()


0. argmin

1. calculating arg min

2. coclusters: 1
3. coherence: inf


4. alpha: 1

% debug % 200

0. argmin

1. calculating arg min

2. coclusters: 1
3. coherence: inf


4. alpha: 1

% debug % 200

0. argmin

1. calculating arg min

2. coclusters: 1
3. coherence: inf


4. alpha: 1

% debug % 200


In [204]:
clusterer = BisectingClusterer(data.T)
clusterer.fit()

array([ 1.,  1.,  1.,  0.,  0.,  1.,  1.])

In [107]:
import glob
glob.glob('TestData/SimulatedDataCoherence/*')

['TestData/SimulatedDataCoherence/LowCoherence.csv',
 'TestData/SimulatedDataCoherence/HighCoherenceMix.csv',
 'TestData/SimulatedDataCoherence/MidCoherence.csv',
 'TestData/SimulatedDataCoherence/RandData.csv',
 'TestData/SimulatedDataCoherence/HighCoherence.csv',
 'TestData/SimulatedDataCoherence/LowCoherenceMix.csv',
 'TestData/SimulatedDataCoherence/MidCoherenceMix.csv']

In [225]:
d = np.array(np.array([0,1,2,3,3,3,6,7,8]))
a = d[np.where(d==3)]

In [226]:
a

array([3, 3, 3])

In [227]:
d

array([0, 1, 2, 3, 3, 3, 6, 7, 8])